In [ ]:
import sys

import IPython.display as ipd
from lightning.pytorch.cli import LightningCLI
import torch
import torchaudio
from unittest.mock import patch

In [ ]:
USE_GPU = False
device = "cuda" if torch.cuda.is_available() and USE_GPU else "cpu"

config = "../lightning_logs/wandb/run-20240419_095659-7vtkk1s1/files/model-config.yaml"
ckpt = "../lightning_logs/wandb/run-20240419_095659-7vtkk1s1/epoch=9-step=312500.ckpt"

with patch.object(
    sys, "argv", ["fit", "-c", str(config), "--trainer.accelerator", device]
):
    cli = LightningCLI(run=False)
    model = cli.model
    synth = cli.datamodule.synth

In [ ]:
state_dict = torch.load(ckpt, map_location=device)["state_dict"]
model.load_state_dict(state_dict)
model = model.eval()

In [ ]:
audio, sr = torchaudio.load("../audio/808_samples/BD 808 Smooth C 09.wav")
if sr != 48000:
    audio = torchaudio.functional.resample(audio, sr, 48000, lowpass_filter_width=512)
    sr = 48000

# Pad to 1 second
audio = torch.nn.functional.pad(audio, (0, sr - audio.shape[1]))
print(audio.shape)

In [ ]:
ipd.Audio(audio.squeeze().numpy(), rate=sr)

In [ ]:
with torch.no_grad():
    params, _, _ = model(audio=audio.to(device))

print(params)
y_hat = synth(torch.clamp(params, 0.0, 1.0))

In [ ]:
ipd.Audio(y_hat.squeeze().cpu().numpy(), rate=sr)

In [ ]:
synth_data = cli.datamodule.test_dataloader()
batch = next(iter(synth_data))

In [ ]:
batch[0].shape

with torch.no_grad():
    params, _, _ = model(audio=batch[0].to(device))


loss = model.loss_fn(params, batch[1].to(device))
print(loss)

y_hat = synth(torch.clamp(params, 0.0, 1.0))

In [ ]:
print(y_hat.shape)
for i in range(16):
    print(batch[0][i].shape)
    print(y_hat[i].shape)
    ipd.display(ipd.Audio(batch[0][i].cpu().numpy(), rate=48000))
    ipd.display(ipd.Audio(y_hat[i].cpu().numpy(), rate=48000))
    print("----")